# Juan Felipe Osorio Franco

In [ ]:
# Required imports
import os
from getpass import getpass
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain, ConversationalRetrievalChain
from langchain.chains.question_answering import load_qa_chain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory
)
from langchain.prompts import PromptTemplate

# Set up OpenAI API key
OPENAI_API_KEY = getpass('Enter your OpenAI API key: ')
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

# Load and process document
loader = PyPDFLoader("ACUERDO_008_2008_CSU_Estatuto_Estudiantil.pdf")
documents = loader.load()

# Split text
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len
)
splits = text_splitter.split_documents(documents)

# Create embeddings and vector store
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(
    documents=splits,
    embedding=embeddings,
    persist_directory="rag_database"
)

# Initialize base LLM
llm = ChatOpenAI(temperature=0)


## Ejercicios:

1. Qué pasa si el elemento de memoria se vuelve muy extenso? En tal caso consultar e implementar:
- Summary Memory.
- Buffer-Summary Memory.

2. Será que todas las preguntas tienen k=5 fragmentos de textos relacionados? Implementar un sistema de RAG Adaptativo; para esto se debe implementar un LLM que filtre los fragmentos de texto que en realidad son relevantes a la hora de responder la pregunta hecha por el usuario.

3. Implementar un Chatbot con memoria a corto plazo que responda preguntas acerca del paper que se estén leyendo.

En caso de que la memoria sea muy grande ariesgamos que el modelo olvide informacion o que choquemos contral el limite de tokens.

In [ ]:
# Create prompt template
template = """
Se te proporcionará una serie de textos que dan instrucciones acerca de como resolver preguntas 
a un estudiante de la universidad Nacional de Colombia. Responde de la manera más completa posible.

Contexto:
{context}

Historial de la conversación:
{chat_history}

Pregunta: {human_input}
Respuesta formal:
"""

prompt = PromptTemplate(
    input_variables=["context", "chat_history", "human_input"],
    template=template
)

# Initialize both memory types
summary_memory = ConversationSummaryMemory(
    llm=llm,
    memory_key="chat_history",
    return_messages=True
)

buffer_summary_memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=2000,
    memory_key="chat_history",
    return_messages=True
)

# Create chains with different memory types
summary_chain = load_qa_chain(
    llm,
    chain_type="stuff",
    memory=summary_memory,
    prompt=prompt
)

buffer_summary_chain = load_qa_chain(
    llm,
    chain_type="stuff",
    memory=buffer_summary_memory,
    prompt=prompt
)

def get_response(question, chain_type="summary"):
    # Choose the appropriate chain
    chain = summary_chain if chain_type == "summary" else buffer_summary_chain
    
    # Get relevant documents
    docs = vectorstore.similarity_search(question, k=5)
    
    # Get response
    response = chain({"input_documents": docs, "human_input": question})
    
    return response["output_text"]

# Example usage
question = "¿Qué es la admisión?"
print("\nRespuesta (usando Summary Memory):")
print(get_response(question, "summary"))
print("\nRespuesta (usando Buffer-Summary Memory):")
print(get_response(question, "buffer"))

In [ ]:
# Create relevance filter prompt
filter_template = """
Evalúa si el siguiente contexto es relevante para responder la pregunta.
Responde únicamente con 'Relevante' o 'No Relevante'.

Pregunta: {question}
Contexto: {context}

Relevancia:"""

filter_prompt = PromptTemplate(
    input_variables=["question", "context"],
    template=filter_template
)

# Create filter chain
filter_chain = LLMChain(
    llm=llm,
    prompt=filter_prompt
)

# Create QA prompt
qa_template = """
Basado en el siguiente contexto relevante, responde la pregunta del estudiante:

Contexto:
{context}

Historial:
{chat_history}

Pregunta: {human_input}
Respuesta formal:"""

qa_prompt = PromptTemplate(
    input_variables=["context", "chat_history", "human_input"],
    template=qa_template
)

# Initialize memory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="human_input"
)

# Create QA chain
qa_chain = load_qa_chain(
    llm,
    chain_type="stuff",
    memory=memory,
    prompt=qa_prompt
)

def get_relevant_documents(question, k=10):
    """Retrieve and filter relevant documents"""
    # Get initial documents
    docs = vectorstore.similarity_search(question, k=k)
    
    relevant_docs = []
    for doc in docs:
        # Check relevance
        result = filter_chain.run(
            question=question,
            context=doc.page_content
        )
        
        if result.strip().lower() == 'relevante':
            relevant_docs.append(doc)
    
    return relevant_docs

def get_adaptive_response(question):
    """Get response using adaptive document retrieval"""
    # Get filtered relevant documents
    relevant_docs = get_relevant_documents(question)
    
    if not relevant_docs:
        return "No encontré información relevante para responder tu pregunta."
    
    # Get response using relevant documents
    response = qa_chain({
        "input_documents": relevant_docs,
        "human_input": question,
        "chat_history": memory
    })
    
    return response["output_text"]

# Example usage
question = "¿Cuáles son los requisitos de admisión?"
print("\nRespuesta adaptativa:")
print(get_adaptive_response(question))

In [ ]:
# Create custom prompt for research paper QA
paper_template = """
Actúa como un asistente experto que está familiarizado con el contenido del artículo de investigación.
Utiliza el siguiente contexto y el historial de la conversación para responder la pregunta de manera clara y precisa.
Si no estás seguro de algo, indícalo honestamente.

Contexto del artículo:
{context}

Historial de la conversación:
{chat_history}

Pregunta: {question}
Respuesta detallada:"""

# Initialize memory for paper chat
paper_memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

# Create the chain for paper QA
paper_qa = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(temperature=0.3),
    retriever=vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 5}
    ),
    memory=paper_memory,
    return_source_documents=True,
    verbose=True
)

def get_paper_response(question):
    """Get response for a research paper question"""
    result = paper_qa({"question": question})
    return {
        "answer": result["answer"],
        "sources": [doc.metadata for doc in result["source_documents"]]
    }

# Example usage
question = "¿Cuál es el principal argumento del documento?"
response = get_paper_response(question)
print("\nRespuesta:", response["answer"])
print("\nFuentes utilizadas:")
for source in response["sources"]:
    print(f"- Página {source.get('page', 'N/A')}")